# 从0.4开始的命名实体识别-序列标注

## 0 前言

  本篇博文以命名实体识别任务为例，介绍通过神经网络完成任务的整体流程，明确任务中各阶段的目标，为初学者提供实现深度学习算法模型的参考，适合了解过一些NLP相关技术，知道理论知识但苦于无从下手的小伙伴。
  区别于如[用深度学习解决nlp中的命名实体识别(ner)问题(深度学习入门项目)](https://cloud.tencent.com/developer/article/1546525)、[命名实体识别NER & 如何使用BERT实现](https://blog.csdn.net/qq_27586341/article/details/103062651)、[huggingface transformers实战系列-04_多语言命名实体识别](https://mp.weixin.qq.com/s/DIV_ACPpjmmr2brVSbqV8w)、[Transformers 库快速入门（六）：序列标注任务](https://xiaosheng.run/2022/03/18/transformers-note-6.html)、[MSRA序列标注实战](https://aistudio.baidu.com/aistudio/projectdetail/3989073)、[基于pytorch的bert_bilstm_crf中文命名实体识别](https://github.com/taishan1994/pytorch_bert_bilstm_crf_ner)等其他命名实体识别相关的优秀文章，本文介绍的重点是构建神经网络解决任务的整体流程，而且不是算法模型各个部分的实现细节，**并未给出解决序列标注问题的常见算法**，希望读者在阅读本文后检索相关资料并自行实现。
  为尽量减少读者在数据处理、词向量等环节的工作量，在阅读本篇博文需要读者对BERT预训练模型有一定的认识，并且了解[transformers库的基本用法](https://www.bilibili.com/video/BV1a44y1H7Jc/?spm_id_from=333.999.0.0&vd_source=3269363961c1c9a10f72f01393a219fd)，这也是标题起做“从0.4开始的命名实体识别”而不是“从0开始的命名实体识别”的原因。最后，由于**笔者能力有限**，所提及的内容**仅供思路上的参考**，请读者**切勿照抄照搬**，对于文中、代码中的错误**恳请各位小伙伴以任意形式斧正**。

## 1任务介绍

命名实体识别（Named Entities Recognition，NER）是完成NLP下游任务的重要步骤之一，如构建知识图谱（Knowledge Graph，KG）、问答系统（Question and Answering System，QA）等。起初，NER的目标旨在从语句中抽取出三个大类实体、七个小类实体。三大类实体分别为“实体类”、“时间类”、“数字类”；七小类包括：“人名”、“地名”、“组织机构名”、“时间”、“日期”、“货币量”、“百分数”。但随着人工智能（Artificial Intelligence，AI）技术的不断推进与工业界需求的不断升级，NER的抽取目标也不再仅限于通用的“三大类”、“七小类”，开始出现了特定领域内的NER任务，如金融、医疗、司法等领域。针对不同领域、不同类型的语料，不同的NER问题需要采用不同的方法来解决。

在NER任务中，实体大致分为三类：连续实体、嵌套实体、非连续实体。对于连续实体，通常采用的序列标注（Sequence Labeling）的方式解决。序列标注，即为句子中每一个字符都进行标注，语句中的实体字符标注实体标记，非实体部分标为“其他”。数据层面上，对于连续型实体的标注方式通常有“BIO”、“BIOES”两种。“B”来自英文“Begin”，意为实体的开始；“I”取自英文“Inner”，意为实体的内部；“O”为“Other”，表示“其他”，用于标注语料中非实体部分；“E”意为“End”，表示实体结束；“S”取自英文“Single”，意为当前字符可独立作为一个实体。比如对于“Person”类型的实体“李时珍”，“李”为起始字，所以其标记应为“B-Person”；“时”既非实体起始位置，也非实体结束位置，故应将其标注为“I-Person”；汉字“珍”是实体的结束位置，当使用BIO标注框架时，其标注为“I-Person”，但若使用BIOES标注框架时，其标注应为“E-Person”。当实体仅由一个汉字组成时，使用BIO标注的结果为“B-类型”，但使用BIOES进行标注的结果为“S-类型”。此外，还有一种“BMES”标注方式，实际上其与"BIOES"标注方式大同小异，其中的“M”与“I”意义相同，对于非实体部分也同样标记为“O”，序列标注的样例如表1所示。

<div align="center">表1 序列标注数据样例</div>

语句|李|时|珍|的|父|亲|李|言|闻
:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
BIO|B-Per|I-Per|I-Per|O|O|O|B-Per|I-Pe|I-Per
BIOES|B-Per|I-Per|E-Per|O|O|O|B-Per|I-Per|E-Per

## 2任务实现

神经网络的实现过程大致可以分为数据处理、模型构建、模型训练、模型推理等几个部分。首先需要读取、观察、清洗数据，然后构建算法模型，实现算法的前向传播计算过程并训练模型，最后使用训练好的模型在真实业务场景中的数据进行预测，以完成任务。

## 2.1数据处理

在开始使用神经网络完成目标任务之前，需要将实验用的数据进行预处理，比如去除文本中的特殊符号、空格符等等，避免与任务无关的语料对模型产生过多影响、甚至导致模型出错。数据预处理的质量依赖于研究人员处理数据的相关经验以及对实验数据的了解程度。
数据集采用1998年人民日报数据集，数据标注如表12所示。

<div align="center">表2 人民日报数据集标注样例</div>

文字|标签
:--:|:--:|
海|O|
钓|O|
比|O|
塞|O|
地|O|
点|O|
在|O|
厦|O|
门|O|
与|O|
金|O|
门|O|
之|O|
间|O|
的|O|
海|O|
域|O|
。|O|

数据处理阶段需要清洗实验数据并准备后续模型所需要的各种特征。读取实验数据过程中，多使用json库或者python自带的open()方法，这里不对其进行过多赘述。

In [1]:
import os
import unicodedata
from tqdm import tqdm
from collections import Counter
import random
import json
import argparse
from torch.utils.data import Dataset,DataLoader
from transformers import AutoTokenizer,BertModel,AdamW
import pandas as pd
import prettytable as pt
import time
import logging
import numpy as np
import torch
from copy import deepcopy
from seqeval.metrics import accuracy_score,f1_score,recall_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
def read_corpus(corpus_path):
    origin_text, origin_label = list(), list()
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()
    sent_, tag_ = [], []
    for line in lines:
        if line != '\n' and  len(line.strip().split())==2:
            [char, label] = line.strip().split()
            sent_.append(char)
            tag_.append(label)
        else:
            origin_text.append(sent_)
            origin_label.append(tag_)
            sent_, tag_ = [], []
    return origin_text,origin_label

#os.getcwd()获取当前代码所在路径
#os.path.join 用来拼接路径，最终得到的data_path为实验数据所在路径
#使用os.path.join()可以避免在linux、windows切换时路径报错
data_path = os.path.join(os.getcwd(),"data","参照组")
#读取数据
train_text,train_label = read_corpus(os.path.join(data_path,"example.train"))
dev_text,dev_label = read_corpus(os.path.join(data_path,"example.dev"))

数据清洗的质量依赖于研究人员处理数据的相关经验以及对实验数据的了解程度。通常需要去除语料中与任务不相关的内容，如空格符、emoji等。同时，要注意观察标签是否存在噪音，以序列标注任务为例，可能由于数据标注人员对标注内容理解不统一，使得相同实体对应不同标签，最终导致训练好的模型性能不佳。
序列标注任务在清洗数据过程中要尤其注意原始文本与标签的长度保持一致。在原始数据中一个汉字对应一个标签，如果在清洗数据时需要剔除文本中的字符，要注意将其与之对应的标签去除，保证文本长度与标签长度相同。
处理过的文本数据和标签以[ [文本1],[文本2],...,[文本n] ]和[ [标注1],[标注2],...,[标注n]]的结构保存，对于普通的序列标注任务来说，后续可以直接以这种形式的数据制作Dataset并封装成DataLoader，但是在进行其他NLP任务时，以这种结构存储数据可能无法满足需求。所以这里我们**绕一圈不必要的弯子**，重新组织一下数据形式。

In [3]:
def sequence_tag2tag(text, label):
    # 在序列标注形式的标签中，寻找实体及其类型和索引
    # 存放实体相关信息，以字典结构保存，其中包括entity、type以及index
    item = dict()
    # 保存当前正在读取的实体，实体结束后会存入item["entity"]中
    _entity = str()
    # ner中存放当前语料包含的所有实体
    ner = list()
    index = list()
    # 遍历序列标注形式的标签，如果当前标签中包含“B-”则表明“上一个实体已经读取完毕，现在开始要开始读取一个新的实体”
    # 如果当前标签中包含“I-”，说明正在读取的实体还未结束，将当前标签所对应的字添加进_entity中，继续遍历
    # 循环结束后，如果item中不为空，说明存在有未保存的实体，将相关实体信息添加到字典中，最后添加到数据集中。
    for i, (t, l) in enumerate(zip(text, label)):
        if "B-" in l:
            if item:
                item["entity"] = _entity
                item["index"] = index
                ner.append(item)
                _entity = str()
                item = dict()
                index = list()
            item["type"] = l.split("-")[1]
            _entity = t
            index.append(i)
        if "I-" in l and item is not None:
            _entity += t
            index.append(i)
    if item:
        item["entity"] = _entity
        item["index"] = index
        ner.append(item)
        _entity = str()
        item = dict()
        index = list()
    return ner

def entity_mask_generating(data_item):
    _data_item = deepcopy(data_item)
    _data_item["sentence"] = list(_data_item["sentence"])
    offset = 0
    # 为数据中的各个实体分配mask
    for ner in _data_item["ner"]:
        ner["mask"] = chr(9830 + offset)
        for idx in ner["index"]:
            _data_item["sentence"][idx] = chr(9830 + offset)
        offset += 1
    data_item["sentence"] = "".join(_data_item["sentence"])
    return _data_item

def data_reform(data_text,data_label):
#     ```
#     将数据转换为以下形式  
#    [
#         {
#             "sentence": ... ，
#             "ner" : [
#		    			{"entity": ... , "type": ... },
#					    {"entity": ... , "type": ... }  
#				      ]
#         },
#         ...,
#         {...}
#     ]
#     ```
    #存放转换形式后的数据
    dataset = list()
    #data_text,data_label代表所有数据和所有标签
    for text,label in zip(data_text,data_label):
        #单条数据
        data = dict()
        data["sentence"] = "".join(text)
        data["ner"] = sequence_tag2tag(text,label)
        dataset.append(entity_mask_generating(data))
    return dataset
train_data = data_reform(train_text,train_label)
train_data = [data for data in train_data if len(data["sentence"]) < 400 and len(data["ner"])]
dev_data = data_reform(dev_text,dev_label)
dev_data = [data for data in dev_data if len(data["sentence"]) < 400 and len(data["ner"])]

此时我们已经将数据处理为：一条数据是一个字典，字典中“sentence”存放文本数据，“ner”中存放文本中出现的实体。在实现其他任务时，还可以在字典里面加入其他特征，比如说词性、相对位置等，也并非一定要拘谨于字典结构，可以根据个人喜好，怎么顺手怎么写。在模型训练过程中，需要将所有任务涉及到的特征都转换成张量形式，所以在数据处理阶段要将所有模型需要的特征准备好。除此之外，在训练过程中还需要计算模型预测与标签之间的“差距”，所以除了特征之外，还需要将标签也存入结构中。

在数据处理阶段不要苦恼将数据处理成什么形式，什么形式都可以。也不用担心是不是多了什么特征、是不是少了什么特征，想起来什么就存什么，后面写着写着发现少了什么，大不了回头补上。就算完全不知道需要用到什么特征，那既然是NLP的相关任务，最起码要有输入的文本对吧？那就先只保存文本，继续完善流程，后面到训练阶段发现缺东西了就回头再补就是，不要担心太多，数据可以变成任何你想要的形状。

为了保证标签能与输入文本相对齐，我们通过“ner”中包含的实体信息来为文本生成标签。data_reform过程中我们使用entity_mask_generating方法为每个实体都单独生成了特殊符号，我们暂时将其称为“mask”。在后续对sentence进行增加、删减字符的过程中，我们会首先将句子中的**实体替换为同长度的mask**，然后对**sentence**进行**修改**，修改完毕后通过**find实体的mask**来**确定**该**实体**的**索引**。通过这种方式清洗数据，可以避免sentence中同一实体出现多次时，每次find只能找到实体第一次出现的位置，但是这种方法无法在包含嵌套实体的数据集中使用。

下面代码块中entity_mask_replacing方法就是返回某条数据将实体替换成其等长mask的功能；index_correctize则是通过某条数据中的mask位置来重新定位各实体的索引，在数据清洗过程中，流程即为：首先遍历数据集拿到单条数据，使用entity_mask_replacing在sentence中把实体替换为相应mask，然后对sentence进行数据清洗操作，最后通过index_correctize方法从清洗过的sentence中重新定位各实体位置。

In [4]:
def entity_mask_replacing(data_item):
    #将sentence中的所有实体替换成相应的mask并返回
    text = list(deepcopy(data_item["sentence"]))
    for ner in data_item["ner"]:
        for idx in ner["index"]:
            text[idx] = ner["mask"]
    text = "".join(text)
    return text
    

def index_correctize(text,data_item):
    #找到data_item中所有实体mask在sentence中的实际位置，并更新ner["index"]
    for ner in data_item["ner"]:
        _entity = ner["entity"]
        _index = text.find(ner["mask"])
        if _index != -1:
            ner["index"] = list(range(_index,_index + len(_entity)))
        else:
              raise Exception(f"{''.join(text)}\ndidn't found{_entity} in the text.\norigin:\n{ner}")
        text = text.replace(ner["mask"]*len(ner["entity"]),ner["entity"])
    data_item["sentence"] = text
    return data_item

## 2.2数据处理补充

在对数据进行清洗、整理之前，需要先观察数据，对数据有一定了解。比如统计文本长度、统计实体类型个数、统计每类实体数量分布等等，这些较为容易实现的操作在本文中不会进行过多赘述。
1. 查看数据中是否有实体对应多种标签
由于数据标注通常都是由多人完成，每个人标注的标准可能存在差异，所以在NER任务中可能存在某个实体对应多种标签，这就属于噪音的一种。这里主要使用Counter库（form collection import Counter），将出现过的实体及其类型以元组的形式保存在集合中，再统计各元组中实体的出现次数，如果出现次数大于1则证明这个实体对应多个实体类型，后续再将标签统一即可。

In [5]:
def noise_entity_finding(data_list):
    entities = set()
    noisy_dict = dict()
    for dataset in data_list:
        for data_item in dataset:
            for ner in data_item["ner"]:
                entities.add((ner["entity"],ner["type"]))
    entities = list(entities)
    entity_appeared = [ent[0] for ent in entities]
    result = dict(Counter(entity_appeared))
    for key,value in result.items():
        if value >= 2 :
            _type = [ent[1] for ent in entities if ent[0]== key]
            noisy_dict[key] = (_type)
    return pd.DataFrame.from_dict(noisy_dict,orient='index')

noise_entity_finding([train_data,dev_data])

,0,1
紫禁城,ORG,LOC
美国,LOC,ORG
哥伦比亚,LOC,ORG
中国,LOC,ORG
黄,LOC,PER
法国,LOC,ORG
罗马尼亚,LOC,ORG
德,LOC,ORG
英格兰,LOC,ORG
布,LOC,PER


2. 全角字符转换为半角字符
由于transformer的tokenizer在处理文本时，会删除某些全角符号，为保证文本与标签数量一致，所以我们需要将所有文本中的全角符号转换成与之相对应的半角符号。

In [6]:
def conver_Q_to_B(data_item):
    def Q2B(uchar):
        """单个字符 全角转半角"""
        inside_code = ord(uchar)
        if inside_code == 0x3000:
            inside_code = 0x0020
        else:
            inside_code -= 0xfee0
        if inside_code < 0x0020 or inside_code > 0x7e: #转完之后不是半角字符返回原来的字符
            return uchar
        return chr(inside_code)
    converted_text = list()
    _text = entity_mask_replacing(data_item)
    for txt in _text:
        _txt = Q2B(txt)
        if len("".join(_txt.split())) >= 1:
            converted_text .append(_txt)
    converted_text = "".join(converted_text) 
    data_item = index_correctize(converted_text,data_item)
    return data_item
for data in train_data:
    data= conver_Q_to_B(data)
for data in dev_data:
    data = conver_Q_to_B(data)

3. 去除文本中不想要的字符

功能实现的方式十分丑陋，这里主要想展示去除unicode特殊符号的方法。另外，涉及到去除字符的操作要注意校正标签中相应位置的值。

In [7]:
def remove_noisy_token(dataset):
    noisy_token = [
    "【",
    "】",
    "&nbsp",
    "/",
    "@",
    "/d",
    ] 
    for data_item in dataset:
        #去除unicode特殊符号
        _text = entity_mask_replacing(data_item)
        sentence = ''.join(c for c in _text if unicodedata.category(c) != 'Co')
        for n in noisy_token:
            sentence = sentence.replace(n,"")
        data_item = index_correctize(sentence,data_item)
    return dataset
train_data = remove_noisy_token(train_data)
dev_data = remove_noisy_token(dev_data)

4.数据增强

NLP中数据增强的方法有很多种，这里只简单概括一些笔者知道的方法，目的也仅仅是为了让读者知道“有这件事儿”，具体的相关知识还需要读者自行了解。

[**AEDA**](https://arxiv.org/abs/2108.13230)

In [8]:
def inert_punctuations(data_item):
    PUNCTUATIONS = ['.', ',', '!', '?', ';', ':']
    _sentence = entity_mask_replacing(data_item)
    _sentence = "".join(_sentence)
    #去除句子中后三分之2的索引
    insertable_index = list(range(len(_sentence) // 3 ))
    #去除句子中实体所在索引
    occupied_index = list(set([i for ner in data_item["ner"] for i in ner["index"] ]))
    for occ in occupied_index:
        try:
            insertable_index.remove(occ)
        except:
            continue
    if len(insertable_index) == 0:
        selected_loc = 0
    else:
        selected_loc = random.sample(insertable_index,1)[0]
    selected_punctuations = PUNCTUATIONS[random.sample(range(len(PUNCTUATIONS)),1)[0]]
    
    new_sentence = str()
    for i,s in enumerate(_sentence):
        if i == selected_loc:
            new_sentence += selected_punctuations
            new_sentence += s
        else:
            new_sentence += s
    data_item = index_correctize(new_sentence,data_item)
    return data_item

new_train_data = list()
for data in train_data:
    new_train_data.append(data)
    new_train_data.append(inert_punctuations(data))
train_data = new_train_data

[**使用相同类型实体替换原实体**](https://zhuanlan.zhihu.com/p/342032812)

这种数据增强方式的实现方式可以参考[An Analysis of Simple Data Augmentation for Named Entity Recognition](https://github.com/abdulmajee/coling2020-data-augmentation/blob/main/augment.py)。直觉上来说，将句子中出现的实体，替换成与其类型相同的实体后，语义应当仍然通顺，以此来达到数据增强的目的。

In [9]:
type2ent = dict()
for dataset in [train_data,dev_data]:
    for data in dataset:
        for ner in data["ner"]:
            current_type = ner["type"]
            if current_type not in type2ent:
                type2ent[current_type] = set()
            type2ent[current_type].add(ner["entity"])
for typ in type2ent:
     type2ent[typ] = list(type2ent[typ])
        
        
def entity_augment(data_item):
    text = list(deepcopy(data_item["sentence"]))

    #将随机替换的实体及相关信息保存到new_ner当中
    new_ner = list()
    offset = 0
    for ner in data_item["ner"]:
        #找到与当前实体同类型的实体索引
        random_entity_index = random.randint(0, len(type2ent[ner["type"] ]) - 1)
        #如果当前存在当前类型实体，且随机得到的实体与原实体不相同，就作为新实体
        if type2ent[ner["type"]]:
            while type2ent[ner["type"]][random_entity_index] == ner["entity"]:
                random_entity_index = random.randint(0, len(type2ent[ner["type"]]))
            random_entity = type2ent[ner["type"]][random_entity_index]
            
            new_ner.append({
                "entity":random_entity
#                 ,"index":list(range(ner["index"][0],ner["index"][0] + len(random_entity)))
                ,"type":ner["type"]
                ,"mask":chr(8830 + offset)
            })
            offset += 1
    #接下来将文本中的原始实体替换为新生成的实体
    #首先把原始实体在原句中替换成相应的mask，
    for ner in data_item["ner"]:
        for idx in ner["index"]:
            text[idx] = ner["mask"]
    text = "".join(text)
    #再把mask换成new_ner中实体的mask，
    for i in range(len(data_item["ner"])):
        entity_mask = data_item["ner"][i]["mask"]*len(data_item["ner"][i]["entity"])
        new_entity_mask = new_ner[i]["mask"]*len(new_ner[i]["entity"])
        text = text.replace(entity_mask,new_entity_mask)
    #重新确定所有mask的索引，把mask替换回实体
    for new in new_ner:
        _index = text.find(new["mask"] * len(new["entity"]))
        if _index != -1:
            new["index"] = list(range(_index , _index + len(new["entity"])))
            text = text.replace(new["mask"] * len(new["entity"]),new["entity"])
        else:
            raise Exception(f"{''.join(text)}\ndidn't found{_entity} in the text.\norigin:\n{ner}")
    return {"sentence":text,"ner":new_ner}

new_train_data = list()
for data in train_data:   
    new_train_data.append(data)
    new_train_data.append(entity_augment(data))
train_data = new_train_data

经历过数据清洗、数据增强等阶段后，就可以为每条数据生成序列标注需要用到的标签。根据实体的长度决定循环次数，第一次执行循环生成的标签应当以“B-”开头，由于使用的是BIO标注框架，所以剩下的循环中生成的标签均以“I-”开头。

In [10]:
def label_generating(data_item):
    text = data_item["sentence"]
    label = ["O"] * len(text)
    for ner in data_item["ner"]:
        for i in range(len(ner["entity"])):
            if i == 0:
                label[ner["index"][i]] = "B-" + ner["type"]
            else:
                label[ner["index"][i]] = "I-" + ner["type"]
    return label
for data in train_data:
    data["label"] = label_generating(data)
for data in dev_data:
    data["label"] = label_generating(data)

为避免重复运行处理数据的代码，可以将处理好的数据保存到本地，本次案例以json形式保存数据。

In [11]:
with open(os.path.join(data_path,"train.json"),"w",encoding="utf-8") as f:
    json.dump(train_data,f,ensure_ascii=False,indent=2)
with open(os.path.join(data_path,"dev.json"),"w",encoding="utf-8") as f:
    json.dump(dev_data,f,ensure_ascii=False,indent=2)

接下来我们要开始把组织好的数据制作成dataloader，以便后面送入模型，config中保存的是各种设置。读者在执行代码时需要将其中的“bert_name”参数改为自己的预训练模型存储路径。

In [12]:
class Config:
    def __init__(self, args):
        with open(args.config, "r", encoding="utf-8") as f:
            config = json.load(f)
        self.loss_type = config["loss_type"]
        self.dataset = config["dataset"]
        self.conv_hid_size = config["conv_hid_size"]
        self.bert_hid_size = config["bert_hid_size"]
        self.dilation = config["dilation"]
        self.epochs = config["epochs"]
        self.batch_size = config["batch_size"]
        self.learning_rate = config["learning_rate"]
        self.bert_learning_rate = config["bert_learning_rate"]
        self.weight_decay = config["weight_decay"]
        
        for k, v in args.__dict__.items():
            if v is not None:
                self.__dict__[k] = v

    def __repr__(self):
        return "{}".format(self.__dict__.items())
    
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./config/chinese_news.json')
parser.add_argument('--save_path', type=str, default='./outputs')
parser.add_argument('--bert_name', type=str, default=r"bert-base-chinese")
parser.add_argument('--device', type=str, default="cuda")
#在notebook中不加这个参数会报错
#pycharm中为：args = parser.parse_args()
args = parser.parse_args(args=[])
config = Config(args)

def get_logger(dataset):
    pathname = "./log/{}_{}.txt".format(dataset, time.strftime("%m-%d_%H-%M-%S"))
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s: %(message)s",
                                  datefmt='%Y-%m-%d %H:%M:%S')

    file_handler = logging.FileHandler(pathname)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)

    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.DEBUG)
    stream_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    return logger

logger = get_logger(config.dataset)
config.logger = logger


模型在预测标签时得到的结果是“数值”，而且送入模型数据的也不能是文本，应该是“数”，所以接下来要做的就是把原始的文本数据，做成“数字”，再封装成DataLoader。不论是语料或者是标签，我们都需要用一个“数字”来代表他们。Transformer中的tokenizer可以帮助我们将语料便捷、高效的转换成“数字”，但是将标签转换为数字这个过程就需要我们自己实现。可以声明一个字典，将每一种标签映射为一个数字，在模型的整个计算中用这个数字来代表这种标签。下面代码块中fill_vocab就是在统计数据中出现了哪些实体类型，将所有出现过的实体类型都映射成一个数字，Vocabulary和fill_vocab的实现如下所示。

In [13]:
def fill_vocab(vocab, dataset):
    entity_num = 0
    for data_item in dataset:
        for tag in data_item["label"]:
            vocab.add_label(tag)
            if "B-" in tag:
                entity_num += 1
    return entity_num
class Vocabulary(object):
    PAD = '<pad>'
    UNK = '<unk>'
    SUC = '<suc>'
    def __init__(self):
        self.label2id, self.id2label = {self.PAD : 0},{0 : self.PAD}

    def add_label(self, label):
        if label not in self.label2id:
            self.label2id[label] = len(self.label2id)
            self.id2label[self.label2id[label]] = label
        assert label == self.id2label[self.label2id[label]]

    def __len__(self):
        return len(self.token2id)

    def label_to_id(self, label):
        return self.label2id[label]

    def id_to_label(self, i):
        return self.id2label[i]
    def save_Vocabulary(self,save_path):
        label2id_path = os.path.join(save_path,"label2id.json")
        id2label_path = os.path.join(save_path,"id2label.json")
        with open(label2id_path,"w",encoding="utf-8") as f:
            json.dump(self.label2id,f,ensure_ascii=False,indent=2)
        with open(id2label_path,"w",encoding="utf-8") as f:
            json.dump(self.id2label,f,ensure_ascii=False,indent=2)
    def load_Vocabulary(self,save_path):
        label2id_path = os.path.join(save_path, "label2id.json")
        id2label_path = os.path.join(save_path, "id2label.json")
        with open(label2id_path,"r",encoding="utf-8") as f:
            self.label2id = json.load(f)
        with open(id2label_path,"r",encoding="utf-8") as f:
            self.id2label = json.load(f)

接下来读取经过预处理的数据，制作Vocabulary，再经过process_bert方法把文本、标签处理成数值形式，存成Dataset类型，最后把Dataset封装成DataLoader。

In [14]:
def load_data_bert(config):
    with open('./data/{}/train.json'.format(config.dataset), 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    with open('./data/{}/dev.json'.format(config.dataset), 'r', encoding='utf-8') as f:
        dev_data = json.load(f)

    tokenizer = AutoTokenizer.from_pretrained(config.bert_name, cache_dir="./cache/")
    vocab = Vocabulary()
    train_ent_num = fill_vocab(vocab, train_data)
    dev_ent_num = fill_vocab(vocab, dev_data)

    table = pt.PrettyTable([config.dataset, 'sentences', 'entities'])
    table.add_row(['train', len(train_data), train_ent_num])
    table.add_row(['dev', len(dev_data), dev_ent_num])
    config.logger.info("\n{}".format(table))
    config.label_num = len(vocab.label2id)
    vocab.save_Vocabulary(os.path.join("./outputs",config.dataset))
    config.vocab = vocab

    train_dataset = NERDataset(*process_bert(train_data, tokenizer, vocab))
    dev_dataset = NERDataset(*process_bert(dev_data, tokenizer, vocab))

    return (train_dataset, dev_dataset), (train_data, dev_data)

process_bert的目标是把数据集中的语料和标签都处理成数值形式以供模型进行计算。首先将语料中的每个字符进行tokenize，然后将经过tonkenize的句子convert成ids，并且在语料起始和语料结束两个位置加上CLS和SEP标签，至此就将语料转换成了数值形式。标签序列也以同样的形式处理，使用BERT时要在原始文本加入[CLS]和[SEP]，要注意将label与原始文本对齐，对齐后使用label_to_id将标签转换成对应的数字即可。

In [15]:
def process_bert(data, tokenizer, vocab):
    bert_inputs = list()
    instance_labels = list()
    sent_length = list()

    for index, instance in enumerate(data):
        tokens = [tokenizer.tokenize(word) for word in instance["sentence"]]
        pieces = [piece for pieces in tokens for piece in pieces]
        _bert_inputs = tokenizer.convert_tokens_to_ids(pieces)
        _bert_inputs = np.array([tokenizer.cls_token_id] + _bert_inputs + [tokenizer.sep_token_id])
        instance["label"] = ["O"] + instance["label"] + ["O"]
        length = len(instance["label"])
        bert_labels = [vocab.label_to_id(tag) for tag in instance["label"]]
        bert_inputs.append(_bert_inputs)
        instance_labels.append(bert_labels)
        sent_length.append(length)
    return bert_inputs, instance_labels, sent_length

接下来就需要将做好的数值形式的数据封装成张量形式，这里就比较容易理解了。在process_bert中处理好的数据，封装成NERDataset类型，在访问其中元素的时候（调用__getitem__），会将其转换成张量类型。

In [16]:
class NERDataset(Dataset):
    def __init__(self, bert_inputs, bert_labels, sent_length):
        self.bert_inputs = bert_inputs
        self.bert_labels = bert_labels
        self.sent_length = sent_length

    def __getitem__(self, item):
        return torch.LongTensor(self.bert_inputs[item]), \
               torch.LongTensor(self.bert_labels[item]), \
               self.sent_length[item]

    def __len__(self):
        return len(self.bert_inputs)

至此处理数据的流程大致就算结束，接下来分别将训练集、验证集、测试集中的数据封装成dataloader。封装时需要设置dataset、batch_size、shuffle等参数，需要特别注意的是需要设置批处理函数collate_fn。在批处理过程中，通常涉及到的操作是padding，即将每个batch中的数据长度统一，任务不同批处理涉及到的操作可能也不同，本次案例的批处理函数实现如下所示。

In [17]:
def collate_fn(data):
    _bert_inputs, _bert_labels, sent_length = map(list, zip(*data))
    batch_size = len(_bert_inputs)
    bert_inputs = torch.zeros(batch_size,max(sent_length),dtype=torch.long)
    bert_labels = torch.zeros(batch_size,max(sent_length),dtype=torch.long)
    def fill(data,new_data):
        for j, x in enumerate(data):
            new_data[j, :x.shape[0]] = x
        return new_data

    bert_inputs = fill(_bert_inputs, bert_inputs)
    bert_labels = fill(_bert_labels, bert_labels)
    sent_length = torch.LongTensor(sent_length)
    return bert_inputs, bert_labels, sent_length

在NERDataset中保存了每条样本的长度，进行批处理时通过获取当前batch中最大的长度，以决定每条样本padding多个少个字符。

In [18]:
datasets, ori_data = load_data_bert(config)
train_loader, dev_loader  = (
        DataLoader(dataset = dataset,
                    batch_size = config.batch_size,
                    collate_fn = collate_fn,
                    shuffle = i == 0,
                    #num_workers=4,
                    drop_last=i == 0)
        for i, dataset in enumerate(datasets)
    )

2022-11-04 15:23:25 - INFO: 
+--------+-----------+----------+
| 参照组 | sentences | entities |
+--------+-----------+----------+
| train  |   11156   |  30348   |
|  dev   |    306    |   854    |
+--------+-----------+----------+


## 2.3模型搭建

封装好的DataLoader中我们只保存了文本内容这一项特征作为模型的输入，最终我们期望得到的结果是每个字符对应所有标签的概率分布。我们先以BERT为基础直接加分类层为例，搭建第一个网络模型，模型代码实现如代码块所示。

In [19]:
class bertLinear(torch.nn.Module):
    def __init__(self,config):
        super(bertLinear,self).__init__()
        self.model_name = "bertLinear"
        self.config = config
        self.bert = BertModel.from_pretrained(config.bert_name,cache_dir="./cache/", output_hidden_states=True)
        self.linear = torch.nn.Linear(in_features=self.config.bert_hid_size,out_features=self.config.label_num)
    def forward(self,bert_inputs):
        try:
            outputs = self.bert(bert_inputs,attention_mask=bert_inputs.ne(0).float())
            sequence_output , cls_output = outputs[0],outputs[1]
            outputs = self.linear(sequence_output)
        except:
            print(f"bert_inputs.shape:{bert_inputs.shape}\nbert_inputs.ne(0).float:{bert_inputs.ne(0).float().shape}")
        
        return outputs
model = bertLinear(config)

模型初始化就包含三个属性，config、bert以及linear。forward方法中是当前模型前向传播的计算过程，我们通过bert得到文本中每个token的词向量，然后将词向量输入分类层得到该词向量对应所有标签的概率分布。
~直接说模型搭建没别的东西了会不会太突兀？~

## 2.4Trainer实现

模型训练怎么实现？模型验证怎么实现？模型推理怎么实现？在哪里都能听到这三个部分的实现，实际上差不多的言论，但是就是一头雾水。~头上没雾水的请配合下表演，谢谢~

这里笔者给出一种理解方式以供参考。模型**训练阶段**负责拟合数据，**完成网络模型参数更新**并输出评测指标显示模型训练效果；模型**验证阶段**负责在验证集上**评测模型**训练效果；模型**推理阶段**负责**对未标注数据进行推理预测**，最后将模型预测出的“数值型”的结果转换成真实标签，也可以叫做“解码”。

**模型训练和模型验证的区别**
模型训练和模型验证过程中都需要输出评测指标查看模型拟合效果。模型训练和模型验证最主要的一个区别在于验证阶段不需要进行参数更新，所以会经常在别人的代码中看到验证阶段的代码都是在with torch.no_grad()所属的缩进中执行的。另外，在别人代码中你一定见到过model.train()和model.eval()这两句代码，其作用是“是否启用模型中的Batch Normalization 和Dropout”，简单来说就是用来控制模型中部分“零件”是否生效，有的“零件”在training过程中需要用到，但是在验证过程中不能用，这就是模型训练和模型验证的区别。

**模型验证和模型推理的区别**
模型验证和模型推理过程中都不要对网络中的参数进行更新，两者的区别在于模型验证过程是有标签的，是可以计算评测指标的，模型推理是在未标注的数据上进行预测的，没有标签不能计算指标，而且还需要将模型预测出的结果转换为真实标签。
这时我们可以在脑海中简单脑补一下各个阶段的代码实现，假设我们有一个Trainer类，里面包含模型各个阶段的实现。

In [20]:
# class Trainer(object):
#     def __init__(self,model):
#         self.model = model
#         #巴拉巴拉
#     def train(self,data_loader):
#         self.model.train()
#         for i, data_batch in enumerate(data_loader):
#             #巴拉巴拉
#         #计算、输出评测指标
#     def eval(self,data_loader):
#         self.model.eval()
#         with torch.no_grad():
#             for i, data_batch in enumerate(data_loader):
#                 #巴拉巴拉
#         #计算输出评测指标
#     def predict(self,data_loader):
#         self.model.eval()
#         with torch.no_grad():
#             for i, data_batch in enumerate(data_loader):
#                 #巴拉巴拉
#             #模型预测结果解码成真实标签

为了结构完整，笔者将模型推理也写在了Trainer中，实际上推理的代码应该在另外一个地方实现。因为推理阶段的数据处理等操作应该是与模型训练、模型验证有所不同的。建议读者可以将预测的代码与训练和验证在不同的地方实现。
基于上述对模型训练、验证、推理的介绍，接下来开始介绍Trainer的具体实现。Trainer中的任何操作都是以model为基础的，所以在初始化Trainer时最先想到的应该就是model，然后想到更新模型参数需要优化器，模型前向传播完成后还需要损失函数计算loss，那么Trainer的__init__函数应当如代码块所示。

In [21]:
# def __init__(self, model):
#     self.model = model
#     criterion = {
#         "ce": torch.nn.CrossEntropyLoss(),
#     }
#     self.criterion = criterion[config.loss_type]
#     bert_params = set(self.model.bert.parameters())
#     other_params = list(set(self.model.parameters()) - bert_params)
#     no_decay = ['bias', 'LayerNorm.weight']
#     params = [
#         {'params': [p for n, p in model.bert.named_parameters() if not any(nd in n for nd in no_decay)],
#          'lr': config.bert_learning_rate,
#          'weight_decay': config.weight_decay},
#         {'params': [p for n, p in model.bert.named_parameters() if any(nd in n for nd in no_decay)],
#          'lr': config.bert_learning_rate,
#          'weight_decay': 0.0},
#         {'params': other_params,
#          'lr': config.learning_rate,
#          'weight_decay': config.weight_decay},
#     ]
#     self.optimizer = AdamW(params, lr=config.learning_rate, weight_decay=config.weight_decay)

接下来开始实现Trainer中的其他功能，首先是模型训练。结合前面的“脑补”，训练部分的逻辑已经很清晰了，访问DataLoader拿到batch数据，输入模型进行计算，然后计算loss，梯度下降方向传播，最后算个评测指标就可以了，实现如代码块所示。

In [22]:
# def train(self,epoch,data_loader):
#     self.model.train()
#     loss_list = list()
#     origin_labels = list()
#     pred_labels = list()
#     #拿batch数据
#     for i,data_batch in tqdm(enumerate(data_loader)):
#         data_batch = [data.to(config.device) for data in data_batch]
#         bert_inputs,bert_labels,sent_length = data_batch
#         #输入模型
#         outputs = model(bert_inputs, bert_labels)
#         #计算损失函数
#         loss = self.criterion(
#                 outputs.view(-1, config.label_num)[(bert_inputs.ne(0).view(-1)) == 1],
#                 bert_labels.view(-1)[(bert_inputs.ne(0).view(-1)) == 1]
#             )
#         #梯度下降反向传播
#         loss.backward()
#         self.optimizer.step()
#         self.optimizer.zero_grad()
#         loss_list.append(loss.cpu().item())
#         #保存输出
#         for origin_label,pred_label,bert_input in zip(bert_labels,outputs,bert_inputs):
#             origin_label = origin_label[bert_input.ne(0).byte()].cpu().numpy()
#             pred_label = torch.argmax(pred_label,-1)[bert_input.ne(0).byte()].cpu().numpy()
#             origin_label = [config.vocab.id_to_label(i) for i in origin_label]
#             pred_label = [config.vocab.id_to_label(i) for i in pred_label]
#             origin_labels.append(origin_label)
#             pred_labels.append(pred_label)
#     #计算指标
#     p = accuracy_score(origin_labels,pred_labels)
#     r = recall_score(origin_labels, pred_labels)
#     f1 = f1_score(origin_labels, pred_labels)
#     table = pt.PrettyTable(["Train {}".format(epoch), "Loss", "F1", "Precision", "Recall"])
#     table.add_row(["Label", "{:.4f}".format(np.mean(loss_list))] +
#                   ["{:3.4f}".format(x) for x in [f1, p, r]])
#     logger.info("\n{}".format(table))

相比模型训练，模型验证只要把model.train()换成model.eval()，并且注意模型运算的期间不需要计算梯度，其实现如下所示。

In [23]:
# def eval(self,epoch,data_loader):
#     self.model.eval()
#     loss_list = list()
#     origin_labels = list()
#     pred_labels = list()
#     with torch.no_grad():
#         #按batch读取数据
#         for i, data_batch in tqdm(enumerate(data_loader)):
#             data_batch = [data.to(config.device) for data in data_batch]
#             bert_inputs, bert_labels, sent_length = data_batch
#             #模型计算
#             outputs = self.model(bert_inputs)
#             #保存原始标签和模型预测出的标签
#             for origin_label,pred_label,bert_input in zip(bert_labels,outputs,bert_inputs):
#                 origin_label = origin_label[bert_input.ne(0).byte()].cpu().numpy()
#                 pred_label = torch.argmax(pred_label,-1)[bert_input.ne(0).byte()].cpu().numpy()
#                 origin_label = [config.vocab.id_to_label(i) for i in origin_label]
#                 pred_label = [config.vocab.id_to_label(i) for i in pred_label]
#                 origin_labels.append(origin_label)
#                 pred_labels.append(pred_label)
#     #计算指标
#     p = accuracy_score(origin_labels,pred_labels)
#     r = recall_score(origin_labels, pred_labels)
#     f1 = f1_score(origin_labels, pred_labels)
#     table = pt.PrettyTable(["Dev {}".format(epoch), "Loss", "F1", "Precision", "Recall"])
#     table.add_row(["Label", "{:.4f}".format(np.mean(loss_list))] +
#                   ["{:3.4f}".format(x) for x in [f1, p, r]])
#     logger.info("\n{}".format(table))
#     return f1

在序列标注任务中，衡量模型性能主要以F1为主，对于训练过程中性能较好的模型要及时保存，通常在训练开始前设置一个很小的数值作为best_f1，如果当前epoch训练出的模型的F1值大于bert_f1，那么就保存这个模型，在后续模型预测时直接装载已保存的模型进行预测即可，模型的保存与装载如代码块所示。

In [24]:
# def save(self, path):
#     torch.save(self.model.state_dict(), path)
# def load(self, path):
#     self.model.load_state_dict(torch.load(path))

至此Trainer的功能基本实现完毕，读者在执行自己任务时可以根据自己的需要对Trainer中的功能进行增减。为保证代码能够顺利运行，我们将上述Trainer的各部分功能组合起来，如代码块所示。

In [27]:
class Trainer(object):
    def __init__(self, model):
        self.model = model
        criterion = {
            "ce": torch.nn.CrossEntropyLoss(),
        }
        self.criterion = criterion[config.loss_type]
        bert_params = set(self.model.bert.parameters())
        other_params = list(set(self.model.parameters()) - bert_params)
        no_decay = ['bias', 'LayerNorm.weight']
        params = [
            {'params': [p for n, p in model.bert.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': config.bert_learning_rate,
             'weight_decay': config.weight_decay},
            {'params': [p for n, p in model.bert.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': config.bert_learning_rate,
             'weight_decay': 0.0},
            {'params': other_params,
             'lr': config.learning_rate,
             'weight_decay': config.weight_decay},
        ]
        self.optimizer = AdamW(params, lr=config.learning_rate, weight_decay=config.weight_decay)
    def train(self, epoch, data_loader):
        self.model.train()
        loss_list = list()
        origin_labels = list()
        pred_labels = list()
        # 拿batch数据
        for i, data_batch in tqdm(enumerate(data_loader)):
            data_batch = [data.to(config.device) for data in data_batch]
            bert_inputs, bert_labels, sent_length = data_batch
            # 输入模型
            outputs = self.model(bert_inputs)
            # 计算损失函数
            loss = self.criterion(
                outputs.view(-1, config.label_num)[(bert_inputs.ne(0).view(-1)) == 1],
                bert_labels.view(-1)[(bert_inputs.ne(0).view(-1)) == 1]
            )
            # 梯度下降反向传播
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            loss_list.append(loss.cpu().item())
            # 保存输出
            for origin_label, pred_label, bert_input in zip(bert_labels, outputs, bert_inputs):
                origin_label = origin_label[bert_input.ne(0).byte()].cpu().numpy()
                pred_label = torch.argmax(pred_label, -1)[bert_input.ne(0).byte()].cpu().numpy()
                origin_label = [config.vocab.id_to_label(i) for i in origin_label]
                pred_label = [config.vocab.id_to_label(i) for i in pred_label]
                origin_labels.append(origin_label)
                pred_labels.append(pred_label)
        # 计算指标
        p = accuracy_score(origin_labels, pred_labels)
        r = recall_score(origin_labels, pred_labels)
        f1 = f1_score(origin_labels, pred_labels)
        table = pt.PrettyTable(["Train {}".format(epoch), "Loss", "F1", "Precision", "Recall"])
        table.add_row(["Label", "{:.4f}".format(np.mean(loss_list))] +
                      ["{:3.4f}".format(x) for x in [f1, p, r]])
        logger.info("\n{}".format(table))
    def eval(self, epoch, data_loader):
        self.model.eval()
#         loss_list = list()
        origin_labels = list()
        pred_labels = list()
        with torch.no_grad():
            for i, data_batch in tqdm(enumerate(data_loader)):
                data_batch = [data.to(config.device) for data in data_batch]
                bert_inputs, bert_labels, sent_length = data_batch
                outputs = self.model(bert_inputs)
                for origin_label, pred_label, bert_input in zip(bert_labels, outputs, bert_inputs):
                    try:
                        origin_label = origin_label[bert_input.ne(0).byte()].cpu().numpy()
                        pred_label = torch.argmax(pred_label, -1)[bert_input.ne(0).byte()].cpu().numpy()
                        origin_label = [config.vocab.id_to_label(i) for i in origin_label]
                        pred_label = [config.vocab.id_to_label(i) for i in pred_label]
                        origin_labels.append(origin_label)
                        pred_labels.append(pred_label)
                    except:
                        print(f"bert_input.shape:{bert_input.shape}\npred_label.shape:{pred_label.shape}\norigin_label:{origin_label}")
        p = accuracy_score(origin_labels, pred_labels)
        r = recall_score(origin_labels, pred_labels)
        f1 = f1_score(origin_labels, pred_labels)
        table = pt.PrettyTable(["Dev {}".format(epoch), "F1", "Precision", "Recall"])
        table.add_row(["Label"] + ["{:3.4f}".format(x) for x in [f1, p, r]])
        logger.info("\n{}".format(table))
        return f1
    def save(self):
        torch.save(
            self.model.state_dict(), os.path.join(config.save_path,config.dataset,self.model.model_name + ".pt")
        )

    def load(self, path=None):
        if path:
            self.model.load_state_dict(torch.load(path))
        else:
            self.model.load_state_dict(
                torch.load(
                    os.path.join(config.save_path, config.dataset, self.model.model_name + ".pt")
                )
            )

## 2.6 模型训练过程实现

各部分功能均已实现完毕，下面开始在调用前面实现好的功能开始训练神经网络。通过load_data_bert读取实验数据并组织成Dataset，然后配置collate_fn和其他参数，将Dataset封装成DataLoader类型，接着实例化神经网络和Trainer工具，最后实现训练过程，每个epoch中先进行一次训练，然后验证模型的训练效果，保存更优的模型参数。

In [28]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./config/chinese_news.json')
parser.add_argument('--save_path', type=str, default='./outputs')
parser.add_argument('--bert_name', type=str, default=r"E:\MyPython\Pre-train-Model\mc-bert-base")
parser.add_argument('--device', type=str, default="cuda")
#在notebook中不加这个参数会报错
#pycharm中为：args = parser.parse_args()
args = parser.parse_args(args=[])
config = Config(args)

logger = get_logger(config.dataset)
logger.info(config)
config.logger = logger

datasets, ori_data = load_data_bert(config)
train_loader, dev_loader = (
    DataLoader(dataset = dataset,
                batch_size = config.batch_size,
                collate_fn = collate_fn,
                shuffle = i == 0,
                #num_workers=4,
                drop_last=i == 0)
    for i, dataset in enumerate(datasets)
)
updates_total = len(datasets[0]) // config.batch_size * config.epochs
logger.info("Building Model")
model = bertLinear(config).to(config.device)
trainer = Trainer(model)

best_f1 = 0
best_test_f1 = 0
#训练config.epochs次
for i in range(config.epochs):
    logger.info("Epoch: {}".format(i))
    #训练模型
    trainer.train(i, train_loader)
    #训练结束验证训练效果
    f1 = trainer.eval(i, dev_loader)
    if f1 > best_test_f1:
        best_f1 = f1
        trainer.save()
logger.info("Best DEV F1: {:3.4f}".format(best_f1))

2022-11-04 20:46:20 - INFO: dict_items([('loss_type', 'ce'), ('dataset', '参照组'), ('conv_hid_size', 96), ('bert_hid_size', 768), ('dilation', [1, 2, 3, 4]), ('epochs', 1), ('batch_size', 2), ('learning_rate', 0.001), ('bert_learning_rate', 5e-05), ('weight_decay', 0), ('config', './config/chinese_news.json'), ('save_path', './outputs'), ('bert_name', 'E:\\MyPython\\Pre-train-Model\\mc-bert-base'), ('device', 'cuda')])
2022-11-04 20:46:20 - INFO: dict_items([('loss_type', 'ce'), ('dataset', '参照组'), ('conv_hid_size', 96), ('bert_hid_size', 768), ('dilation', [1, 2, 3, 4]), ('epochs', 1), ('batch_size', 2), ('learning_rate', 0.001), ('bert_learning_rate', 5e-05), ('weight_decay', 0), ('config', './config/chinese_news.json'), ('save_path', './outputs'), ('bert_name', 'E:\\MyPython\\Pre-train-Model\\mc-bert-base'), ('device', 'cuda')])
2022-11-04 20:46:20 - INFO: dict_items([('loss_type', 'ce'), ('dataset', '参照组'), ('conv_hid_size', 96), ('bert_hid_size', 768), ('dilation', [1, 2, 3, 4]), ('

## 2.5Predictor实现

推理阶段中数据处理与训练阶段大致相同，但由于原始测试集中包含了文本及标签，为模拟无标签的场景，故对于测试集的数据仅做了最基本的处理，实现如下所示。

In [ ]:
test_text,test_label = read_corpus(os.path.join(data_path,"example.test"))
test_data = [{"sentence":"".join(data)} for data in test_text if len(data) <= 400]
with open(os.path.join(data_path,"test.json"),"w",encoding="utf-8") as f:
    json.dump(test_data,f,ensure_ascii=False,indent=2)

笔者认为模型预测部分不应当与训练、验证等方法在同一个地方实现，故在此另起新类“Predictor”，并在其中完成模型预测的相关代码实现。模型预测阶段不需要计算指标，只要注意将模型预测出的结果转换为真实标签即可。由于序列标注形式的标签难以观察抽取效果，所以使用sequence_tag2tag方法将序列标注的结果组织成训练集“ner”，然后再进行保存、展示。后续具体的制作Dataset、封装Dataloader，批处理、及推理过程等步骤请读者自行实现。

In [ ]:
class Predictor(object):
    def __init__(self, model):
        self.model = model
    def load(self, path):
        self.model.load_state_dict(torch.load(path))

    def sequence_tag2tag(self, text, label):
        # 统计序列标注中的实体及其类型
        # 存放实体相关信息，以字典结构保存，其中包括entity、type以及index
        item = dict()
        # 保存当前正在读取的实体，实体结束后会存入item["entity"]中
        _entity = str()
        # ner中存放当前语料包含的所有实体
        ner = list()
        index = list()
        # 遍历序列标注形式的标签，如果当前标签中包含“B-”则表明“上一个实体已经读取完毕，现在开始要开始读取一个新的实体”
        # 如果当前标签中包含“I-”，说明正在读取的实体还未结束，将当前标签所对应的字添加进_entity中，继续遍历
        # 循环结束后，如果item中不为空，说明存在有未保存的实体，将相关实体信息添加到字典中，最后添加到数据集中。
        for i, (t, l) in enumerate(zip(text, label)):
            if "B-" in l:
                if item:
                    item["entity"] = _entity
                    item["index"] = index
                    ner.append(item)
                    _entity = str()
                    item = dict()
                    index = list()
                item["type"] = l.split("-")[1]
                _entity = t
                index.append(i)
            if "I-" in l and item is not None:
                _entity += t
                index.append(i)
        if item:
            item["entity"] = _entity
            item["index"] = index
            ner.append(item)
            _entity = str()
            item = dict()
            index = list()
        return ner
    def predcit(self,data_loader,origin_data):
        result = list()
        batch = 0
        with torch.no_grad():
            for data_batch in tqdm(data_loader):
                sentence_batch = origin_data[batch : batch + args.batch_size]
                data_batch = [data.to(args.device) for data in data_batch]
                bert_inputs, sent_length = data_batch
                outputs = self.model(bert_inputs)
                for sentence, pred_label, bert_input in zip(sentence_batch,outputs,bert_inputs):
                    sentence = sentence["sentence"]
                    pred_label = torch.argmax(pred_label,-1)[bert_input.ne(0).byte()].cpu().numpy()
                    pred_label = [args.vocab.id_to_label(str(i)) for i in pred_label]
                    result.append({"sentence":sentence,"label":self.sequence_tag2tag(sentence,pred_label)})
                batch += args.batch_size
        with open(os.path.join(args.save_path,args.task,"model_predicted.json"), "w", encoding="utf-8") as f:
            json.dump(result, f, indent=2, ensure_ascii=False)

# 3.结束语

希望本文可以帮助读者梳理神经网络的构建流程、消除刚开始接触人工智能的小伙伴对代码实现的“恐惧”。对于其他微调的任务，希望读者可以做到自行修改process_bert方法、自己定义任务所需的Dataset和collate_fn、替换网络模型等，以独立完成任务目标。另外，本文的py形式代码详见[示例代码](https://github.com/Antiqueeeee/Flat-ner-baseline)，其中包含了推理阶段的相关实现，供读者参考。受限于笔者的自身水平，文中、代码中**一定存在疏漏和错误**，请读者切记**不要照抄照搬**，并对文中的错误进行指正。